# Rust Develop

Rust-specific development routine. Delegates implementation to an AI
assistant, builds and tests the result, then hands build/test output
to a QA engineer AI to diagnose and fix any failures. Use this routine
for Rust projects where you want cargo build --release and cargo test
as the verification gate with automated fix-up.

In [ ]:
spec_file = ""
message_file = ""
message_id = ""
message_dir = ""
chain = ""
seq = ""

In [ ]:
import os

if spec_file and os.path.isfile(spec_file):
    WORK_FILE = spec_file
else:
    WORK_FILE = message_file

In [ ]:
%%bash -s "$WORK_FILE"
claude -p "You are a senior Rust engineer. Read the work description at $1.
Implement all requirements. Follow Rust best practices: proper error
handling with Result/Option, no unwrap in library code, idiomatic types,
and tests where appropriate. Work methodically through each requirement." \
  --allowedTools 'Edit,Write,Bash(cargo*),Bash(npm*),Bash(python*),Bash(make*)'

In [ ]:
%%bash -s "$message_dir"
set -euo pipefail
echo "=== Building (release) ==="
cargo build --release 2>&1 | tee "$1/build.log" || true

echo "=== Running tests ==="
cargo test 2>&1 | tee "$1/test-output.log" || true

In [ ]:
%%bash -s "$WORK_FILE" "$message_dir"
claude -p "You are an expert Quality Assurance Engineer for a Rust project.

Read the work description at $1.
Read the build output at $2/build.log.
Read the test output at $2/test-output.log.

If the build or tests passed cleanly, verify the implementation meets all
requirements and acceptance criteria from the work description — exit 0.

If there are build errors or test failures:
1. Diagnose each failure — read the relevant source and test code.
2. Apply the minimal fix. Prefer fixing source code over weakening tests.
3. Do NOT add new features or refactor unrelated code.
4. Run cargo build --release and cargo test again to confirm everything passes.
Exit 0 only if the build succeeds and all tests pass." \
  --allowedTools 'Edit,Write,Bash(cargo*),Bash(npm*),Bash(python*),Bash(make*)'